# Case study 1: flu shot encouragement (logistic)
This notebook includes experiments from Case Study 1 from the paper Multi-Source Causal Inference Using Control Variates. Specifically, this notebook contains experiments using the logistic model to estimate the ATE and odds ratios.

We use flu shot data from Section 8.1 of [Ding and Lu 2016](https://www.dropbox.com/s/jxk76wk8ckxx4m3/Ding_et_al-2017%20JRSSB%20Principal%20stratification%20analysis%20using%20principal%20scores.pdf?dl=0). The original dataset fludata.txt can be downloaded at https://rss.onlinelibrary.wiley.com/hub/journal/14679868/series-b-datasets/79_3a

The variables are:

- Z: the binary randomized encouragement to get the flu shot
- Y: the binary outcome of flu-related hospitalization. 
- X: all covariates. Most of them are binary. 



In [1]:
import numpy as np
import pandas as pd
from importlib import reload

import data_sampler
import bootstrap

In [8]:
df = pd.read_csv('fludata.txt', sep=" ")

In [11]:
Y_COLUMN = 'outcome'
Z_COLUMN = 'assign'
X_COLUMNS = ['age', 'copd', 'dm', 'heartd', 'race', 'renal', 'sex', 'liverd']

# Data generation using logistic regression model with interaction terms

In this section, we assume that the data generating outcome model is

$$P(Y=1 | Z = z, X = x) = \frac{e^{\beta_0 + \beta_1 z + \beta_2 ^T x + \beta_3 ^T xz}}{1 + e^{\beta_0 + \beta_1 z + \beta_2^T x + \beta_3 ^T xz}}$$

This allows for linear heterogenous effects in $x$.

## Fit model to get P(Y = 1 | Z = z, X = x)

In [6]:
data_sampler_interaction_logistic = data_sampler.DataSamplerInteractionLogistic(Z_COLUMN, X_COLUMNS, Y_COLUMN)
data_sampler_interaction_logistic.fit_outcome(df, print_results=True)

Training accuracy for outcome model: 0.915065
Training AUC for outcome model: 0.666010
Coefficients for outcome model: [[ 9.68703784e-01 -6.16308889e-03  4.39076268e-01  3.34411150e-01
   8.32099888e-01  1.30415691e-02  1.34967584e+00  9.08488042e-02
  -3.71535058e+00 -3.91651689e-03 -2.02438589e-01  2.73969047e-01
  -3.58542569e-01 -5.37481299e-01  3.67910944e-01 -6.42304872e-01
   4.79688067e+00]]


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='none',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Fit model to get $P(Z = 1 | X = x)$ (propensity score)

We assume that the propensity score comes from a simple logistic regression model: 

$$P(Z = 1 | X = x) = \frac{e^{a_0 + a_1^Tx}}{ 1 + e^{a_0 + a_1^Tx}}$$

We fit $a_0, a_1$ from the data.

In [13]:
data_sampler_interaction_logistic.fit_propensity(df, print_results=True)

Training accuracy for propensity model: 0.526389


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='none',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Generate case control data

In [14]:
selection_biased_samples = data_sampler_interaction_logistic.generate_selection_biased_data(df, num_samples=10000)
selection_biased_samples.describe()

Generated 100000 samples before selection bias
Filtered to 16788 samples after selection bias; only returning the requested 10000


,age,copd,dm,heartd,race,renal,sex,liverd,assign,outcome
count,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,64.922900,0.31400,0.309300,0.635800,0.626600,0.029000,0.636500,0.003300,0.492700,0.458800
std,12.533318,0.46414,0.462228,0.481229,0.483731,0.167815,0.481031,0.057354,0.499972,0.498325
min,14.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,59.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,67.000000,0.00000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,73.000000,1.00000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,100.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Compute ATE estimates with and without control variate

In [ ]:
def ATE_estimator_fn_interaction(df_input):
    data_sampler_interaction_logistic = data_sampler.DataSamplerInteractionLogistic(Z_COLUMN, X_COLUMNS, Y_COLUMN)
    data_sampler_interaction_logistic.fit_outcome(df_input)
    return data_sampler_interaction_logistic.get_ATE_estimate(df_input)

def CV_estimator_fn_interaction(df_input_obs, df_input_bias):
    data_sampler_interaction_logistic = data_sampler.DataSamplerInteractionLogistic(Z_COLUMN, X_COLUMNS, Y_COLUMN)
    OR_xs = df_input_obs[X_COLUMNS] # Average over all xs in the observational dataset.
    # Estimate OR from observational dataset
    data_sampler_interaction_logistic.fit_outcome(df_input_obs)
    OR_obs = np.mean(data_sampler_interaction_logistic.get_conditional_OR_estimates(OR_xs))
    # Estimate OR from selection bias dataset
    data_sampler_interaction_logistic.fit_outcome(df_input_bias)
    OR_bias = np.mean(data_sampler_interaction_logistic.get_conditional_OR_estimates(OR_xs))
    return OR_obs - OR_bias

CV_samples, ATE_hat_samples, _ = bootstrap.run_bootstrap_df(df_obs=df, 
              df_bias=selection_biased_samples, 
              n_replicates=300, 
              ATE_estimator_fn=ATE_estimator_fn_interaction,
              CV_estimator_fn=CV_estimator_fn_interaction,
             )

In [34]:
sample_cov = np.cov(np.array([ATE_hat_samples, CV_samples]), ddof=1)

# Get optimal control variates coefficient
cov_ATE_CV = sample_cov[0][1]
var_CV = sample_cov[1][1]
optimal_CV_coeff = cov_ATE_CV / var_CV
print("optimal CV coefficient:", optimal_CV_coeff)

optimal CV coefficient: 0.057495495362146105


In [36]:
# Get variance/bias of ATE estimators with and without CV.
CV_samples, ATE_hat_samples, ATE_hat_CV_samples = bootstrap.run_bootstrap_df(
    df_obs=df, 
    df_bias=selection_biased_samples, 
    n_replicates=300, # Try increasing this
    ATE_estimator_fn=ATE_estimator_fn_interaction,
    CV_estimator_fn=CV_estimator_fn_interaction,
    optimal_CV_coeff=optimal_CV_coeff)

ATE_var = np.var(np.array(ATE_hat_samples), ddof=1)
print(">>> Variance of ATE estimator:", ATE_var)

ATE_bias = np.mean(np.array(ATE_hat_samples)) - ATE_estimate
print(">>> Bias of ATE estimator:", ATE_bias)

ATE_CV_var = np.var(np.array(ATE_hat_CV_samples), ddof=1)
print(">>> Variance of ATE estimator with CV:", ATE_CV_var)

ATE_CV_bias = np.mean(np.array(ATE_hat_CV_samples)) - ATE_estimate
print(">>> Bias of ATE estimator with CV:", ATE_CV_bias)

>>> Variance of ATE estimator: 0.000102341575760821
>>> Bias of ATE estimator: 0.0004283211889470496
>>> Variance of ATE estimator with CV: 2.9515768379399486e-05
>>> Bias of ATE estimator with CV: 0.0006431299551389984
